## Features Extraction
To provide the model with the capabilities of understanding the context and detect similar words, instead of using a simple bag of words representation, we opted for a word embedding approach. In particular, we used the Word2Vec model provided by the Gensim library. The model is trained on the train set and then used to transform the reviews in a vector representation.
The model specification is:
- **vector_size**: 30 and 150
- **window**: 5
- **min_count**: 2

In [7]:
import pymongo as pm
import pandas as pd
import numpy as np
import gensim
import sklearn as sk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /home/davide/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1. Data Loading

In [8]:
client = pm.MongoClient('mongodb://localhost:27017/')
spark_db = client['spark_db']
books_ratings = spark_db['book_reviews']

### 2. Data Reshaping
The goal is to predict the helpfulness of a review based on its content. Thus all the sample without a text or with no helpfulness votes are removed.
To take into account both the number of helfpful votes and the total votes, a laplacian smoothing has been used. In particular the helpfulness score to predict is computed as follows:
$$helpfulness\_score = (\frac{helpful\_votes + smoothing\_param}{total\_votes + 2*smoothing\_param})$$

Thereby the helpfulness score is in the range [0,1], specifically the score tends to the natural ratio when the number of votes increases. 

In [9]:
pipeline_remove = {'$match':{
                        'review/score':{'$exists':True},
                        'N_helpful'	:{'$exists':True, '$ne':0},
                        'Tot_votes'	:{'$exists':True, '$ne':0}
                        }
    
                }

smoothing_param = 1

pipeline_project = {'$project':{
                            'review/text':1,
                            'helpfulness_score':{'$divide':[
                                                        {'$sum':['$N_helpful', smoothing_param]},
                                                        {'$sum': ['$Tot_votes', smoothing_param*2]}
                                                             ]
                                                 },
                            '_id':0,
                            }
                    }

mongo_dataset = books_ratings.aggregate([pipeline_remove, pipeline_project])
df_dataset = pd.DataFrame(list(mongo_dataset))
arr_dataset = np.array(df_dataset)

X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(arr_dataset[:,0], arr_dataset[:,1], test_size=0.2, random_state=42)

In [10]:
stop_words = set(stopwords.words('english'))

def preprocess(doc):
    tokens = gensim.utils.simple_preprocess(doc)
    return [token for token in tokens if token not in stop_words]

X_train_w2v = [preprocess(doc) for doc in X_train]

In [11]:
model = gensim.models.Word2Vec(X_train_w2v, vector_size=150, window=5, min_count=2)

model.save('_gitignore/word2vec.model')

def get_embedding(doc):
    embeddings = []
    words = preprocess(doc)
    for word in words:
        if word in model.wv:
            embeddings.append(model.wv[word])
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)

X_train_embedding = [get_embedding(doc) for doc in X_train]
X_test_embedding = [get_embedding(doc) for doc in X_test]

In [12]:
# Store the embedding
np.savez('_gitignore/train_data_wv2_150_5.npz',x = X_train_embedding, y = Y_train)
np.savez('_gitignore/test_data_w2v_150_5.npz',x = X_test_embedding, y = Y_test)